# Machine Learning Assignment 8


## 1. Data Processing:


In [8]:
# a) Import the following libraries:
import sys
import os
import json
import pandas
import numpy
import optparse

from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dense, Dropout, SimpleRNN
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from collections import OrderedDict
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot
%matplotlib inline
from keras.preprocessing.image import ImageDataGenerator

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
# b) lets import the data
dataset = pd.read_csv("/content/drive/My Drive/Assignment/dev-access.csv", engine='python', quotechar='|', header=None)
dataset.head()

,0,1
0,"{""timestamp"":1502738402847,""method"":""post"",""qu...",0
1,"{""timestamp"":1502738402849,""method"":""post"",""qu...",0
2,"{""timestamp"":1502738402852,""method"":""post"",""qu...",0
3,"{""timestamp"":1502738402852,""method"":""post"",""qu...",0
4,"{""timestamp"":1502738402853,""method"":""post"",""qu...",0


In [11]:
# c) We then need to convert to a numpy.ndarray type: 
dataset = dataset.values

In [12]:
# d) lets check the shape of our data, it should be (26773, 2).
dataset.shape

(26773, 2)

In [13]:
dataset

array([['{"timestamp":1502738402847,"method":"post","query":{},"path":"/login","statusCode":401,"source":{"remoteAddress":"88.141.113.237","referer":"http://localhost:8002/enter"},"route":"/login","headers":{"host":"localhost:8002","accept-language":"en-us","accept-encoding":"gzip, deflate","connection":"keep-alive","accept":"*/*","referer":"http://localhost:8002/enter","cache-control":"no-cache","x-requested-with":"XMLHttpRequest","content-type":"application/json","content-length":"36"},"requestPayload":{"username":"Carl2","password":"bo"},"responsePayload":{"statusCode":401,"error":"Unauthorized","message":"Invalid Login"}}',
        0],
       ['{"timestamp":1502738402849,"method":"post","query":{},"path":"/login","statusCode":401,"source":{"remoteAddress":"88.141.113.237"},"route":"/login","headers":{"host":"localhost:8002","connection":"keep-alive","cache-control":"no-cache","accept":"*/*","accept-encoding":"gzip, deflate, br","accept-language":"en-US,en;q=0.8,es;q=0.6","content-t

In [14]:
#e) Store all rows and the 0th index as the feature data:
X = dataset[:,0]

In [15]:
#f) Store all rows and index 1 as the target variable: 
Y = dataset[:,1]
Y

array([0, 0, 0, ..., 1, 1, 1], dtype=object)

In [16]:
#g) we will clean our data, and remove unwanted variables, which are timestamp, headers, source, route, responsePayload
for index, item in enumerate(X):
    # Quick hack to space out json elements
    reqJson = json.loads(item, object_pairs_hook=OrderedDict)
    del reqJson['timestamp']
    del reqJson['headers']
    del reqJson['source']
    del reqJson['route']
    del reqJson['responsePayload']
    X[index] = json.dumps(reqJson, separators=(',', ':'))

In [17]:
#h) We next will tokenize our data, which just means vectorizing our text.
tokenizer = Tokenizer(filters='\t\n', char_level=True)
tokenizer.fit_on_texts(X)

num_words = len(tokenizer.word_index)+1
X = tokenizer.texts_to_sequences(X)

In [18]:
#i) Need to pad our data as each observation has a different length
max_log_length = 1024
X_processed = sequence.pad_sequences(X, maxlen=max_log_length)


In [19]:
# j) Create your train set to be 75% of the data and your test set to be 25
from sklearn.model_selection import train_test_split
#---Splitting the data into train and test----#
X_train, X_test, y_train, y_test = train_test_split(X_processed, Y, test_size = 0.25, random_state = 42)

## 2. Model 1 - RNN

In [20]:
#a) Initialising the RNN
classifier = Sequential()

In [21]:
# b) add an Embedding Layer
# c) add a SimpleRNN layer
# d) add a Dense layer


classifier.add(Embedding(num_words, 32, input_length=max_log_length))
classifier.add(SimpleRNN(units = 32,activation="relu"))
classifier.add(Dense(units=1, activation = "relu"))

In [22]:
# e) lets compile the model
classifier.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'binary_crossentropy')

In [23]:
# f) model summary
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1024, 32)          2016      
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 32)                2080      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 4,129
Trainable params: 4,129
Non-trainable params: 0
_________________________________________________________________


In [24]:
# Converting the X_train and y_train to float otherwise the model.fit gives error
X_train = np.asarray(X_train).astype(np.float32)
y_train = np.asarray(y_train).astype(np.float32)

In [25]:
# g) Fitting the RNN to the Training set
classifier.fit(X_train, y_train, validation_split = 0.25, epochs = 3, batch_size = 128)

Epoch 1/3
118/118 [==============================] - 37s 313ms/step - loss: 7.7437 - accuracy: 0.4980 - val_loss: 7.7709 - val_accuracy: 0.4962
Epoch 2/3
118/118 [==============================] - 37s 312ms/step - loss: 7.7437 - accuracy: 0.4980 - val_loss: 7.7709 - val_accuracy: 0.4962
Epoch 3/3
118/118 [==============================] - 37s 310ms/step - loss: 7.7437 - accuracy: 0.4980 - val_loss: 7.7709 - val_accuracy: 0.4962


In [26]:
# Converting the X_test and y_test to float otherwise the model.fit gives error
X_test = np.asarray(X_train).astype(np.float32)
y_test = np.asarray(y_train).astype(np.float32)

In [27]:
# h) Use the .evaluate() method to get the loss value & the accuracy value on the test data. 
classifier.evaluate(X_test, y_test, batch_size = 128)

157/157 [==============================] - 11s 68ms/step - loss: 7.7505 - accuracy: 0.4975


[7.750502109527588, 0.49753475189208984]

## 3) Model 2 - LSTM + Dropout Layers:

In [28]:
# a)Initialising the RNN
classifier_LSTM = Sequential()
#add an Embedding Layer 
classifier_LSTM.add(Embedding(num_words, 32, input_length=max_log_length))
#Adding the first LSTM layer and some Dropout regularisation
classifier_LSTM.add(LSTM(units = 64, recurrent_dropout = 0.5))
classifier_LSTM.add(Dropout(0.5))
#add a Dense layer
classifier_LSTM.add(Dense(units=1, activation = "relu"))

In [29]:
#b)Compiling the RNN
classifier_LSTM.compile(optimizer = 'adam', metrics = ["accuracy"], loss = 'binary_crossentropy')

In [30]:
#c)Model summary
classifier_LSTM.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total params: 26,913
Trainable params: 26,913
Non-trainable params: 0
_________________________________________________________________


In [31]:
#d)Use the .fit() method to fit the model on the train data. Use a validation split of 0.25, epochs=3 and batch size = 128.
classifier_LSTM.fit(X_train, y_train, validation_split = 0.25, epochs = 3, batch_size = 128)

Epoch 1/3
118/118 [==============================] - 226s 2s/step - loss: 0.8538 - accuracy: 0.5379 - val_loss: 0.5465 - val_accuracy: 0.6697
Epoch 2/3
118/118 [==============================] - 231s 2s/step - loss: 0.4721 - accuracy: 0.8177 - val_loss: 0.2585 - val_accuracy: 0.9382
Epoch 3/3
118/118 [==============================] - 225s 2s/step - loss: 0.2651 - accuracy: 0.9325 - val_loss: 0.2187 - val_accuracy: 0.9090


In [32]:
#e)Use the .evaluate() method to get the loss value & the accuracy value on the test data. 
classifier_LSTM.evaluate(X_test, y_test, batch_size = 128)

157/157 [==============================] - 50s 316ms/step - loss: 0.2366 - accuracy: 0.9044


[0.23663170635700226, 0.9044275283813477]

## 4) Recurrent Neural Net Model 3: Build Your Own

### a) RNN Requirements



In [33]:
# Initialising the RNN
custom_classifier = Sequential()
# add an Embedding Layer 
custom_classifier.add(Embedding(num_words, 32, input_length=max_log_length))
# Adding the first LSTM layer and some Dropout regularisation
custom_classifier.add(LSTM(units = 128, recurrent_dropout = 0.5, return_sequences=True))
custom_classifier.add(Dropout(0.25))
# Adding the first LSTM layer and some Dropout regularisation
custom_classifier.add(LSTM(units = 64, recurrent_dropout = 0.5, return_sequences=True))
custom_classifier.add(Dropout(0.25))
# Adding a second LSTM layer and some Dropout regularisation
custom_classifier.add(LSTM(units = 64, recurrent_dropout = 0.5, return_sequences=False))
custom_classifier.add(Dropout(0.5))
# add a Dense layer
custom_classifier.add(Dense(units=1, activation = "relu"))

### b) Compiler Requirements



In [34]:
#stochastic gradient descent as the optimizer for this model
custom_classifier.compile(optimizer = 'SGD', metrics = ['accuracy'], loss = 'binary_crossentropy')

### c) Print the model summary



In [35]:
custom_classifier.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 1024, 32)          2016      
_________________________________________________________________
lstm_1 (LSTM)                (None, 1024, 128)         82432     
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024, 128)         0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024, 64)          49408     
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024, 64)          0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)               

### d) Use the .fit() method to fit the model on the train data.



In [36]:
# Use a validation split of 0.25, epochs=3 and batch size = 128.
custom_classifier.fit(X_train, y_train, validation_split = 0.25, epochs = 3, batch_size = 128)

Epoch 1/3
118/118 [==============================] - 1097s 9s/step - loss: 7.5668 - accuracy: 0.5026 - val_loss: 7.5669 - val_accuracy: 0.5038
Epoch 2/3
118/118 [==============================] - 1098s 9s/step - loss: 7.5937 - accuracy: 0.5020 - val_loss: 7.5669 - val_accuracy: 0.5038
Epoch 3/3
118/118 [==============================] - 1098s 9s/step - loss: 7.5937 - accuracy: 0.5020 - val_loss: 7.5669 - val_accuracy: 0.5038


### e) Use the .evaluate() method to get the loss value & the accuracy value on the test data



In [37]:
# Evaluate on the test data with a batch size of 128 
custom_classifier.evaluate(X_test, y_test, batch_size = 128)

157/157 [==============================] - 212s 1s/step - loss: 7.5870 - accuracy: 0.5025


[7.587025165557861, 0.5024652481079102]

## Conceptual Questions:


#### 5) Explain the difference between the relu activation function and the sigmoid activation function.



###### Sigmoid function
The Sigmoid Function curve looks like a S-shape.

   <img src = "Screen Shot 2020-12-03 at 6.14.16 PM.png" width="400">

The sigmoid function exists between (0 to 1). Therefore, it is especially used for models where we have to predict the probability as an output.Since probability of anything exists only between the range of 0 and 1, sigmoid is the right choice.The function is differentiable.That means, we can find the slope of the sigmoid curve at any two points. The function is monotonic but function’s derivative is not.The logistic sigmoid function can cause a neural network to get stuck at the training time.


###### ReLU (Rectified Linear Unit) Activation Function

   <img src = "Screen Shot 2020-12-03 at 6.14.53 PM.png" width="300">
    
The ReLU is the most used activation function in the world right now.Since, it is used in almost all the convolutional neural networks or deep learning.

As you can see, the ReLU is half rectified (from bottom). f(z) is zero when z is less than zero and f(z) is equal to z when z is above or equal to zero.

Range: [0 to infinity)

The function and its derivative both are monotonic.
But the issue is that all the negative values become zero immediately which decreases the ability of the model to fit or train from the data properly. That means any negative input given to the ReLU activation function turns the value into zero immediately in the graph, which in turns affects the resulting graph by not mapping the negative values appropriately.

#### 6) Describe what one epoch actually is (epoch was a parameter used in the .fit() method).


The number of epochs is a hyperparameter that defines the number times that the learning algorithm will work through the entire training dataset.

One epoch means that each sample in the training dataset has had an opportunity to update the internal model parameters. An epoch is comprised of one or more batches. For example, as above, an epoch that has one batch is called the batch gradient descent learning algorithm.

#### 7) Explain how dropout works (you can look at the keras code and/or documentation) for (a) training, and (b) test data sets.


Dropout is a regularization method that approximates training a large number of neural networks with different architectures in parallel.

During training, for each hidden layer, for each training sample, for each iteration, ignore a random fraction, p, number of nodes and corresponding activations. Dropout changed the concept of learning all the weights together, to learning a fraction of the weights in the network in each training iteration.This means that their contribution to the activation of downstream neurons is temporally removed on the forward pass and any weight updates are not applied to the neuron on the backward pass. This prevents units from co-adapting too much. 

In the testing phase, dropout uses all all activations, but reduces them by a factor p, to account for the missing activations during training. The reason why we scale the weights w by p during the testing phase is because the expected value of a Dropout network is equivalent to a regular network with its weights scaled with the Dropout rate p. The scaling makes the inferences from a Dropout network comparable to the full network.

#### 8) Explain why problems such as this homework assignment are better modeled with RNNs than CNNs. What type of problem will CNNs outperform RNNs on?

The reason why the homework assignment is better modeled with RNN than CNN is because of Recurrent Neural Networks(RNN). RNNs and LSTMs by design are most suitable for working with seuqence prediction problems. The reason being, the ability to process temporal information or data that comes in sequences, such as a sentence for example. And on other hand, CNNs defining feature is that it performs the convolution operation in certain layers – hence, the name Convolutional Neural Network.

The Convolutional Neural Networks, or CNNs, CNNs are great at interpreting visual data and data that does not come in a sequence. However, they fail in interpreting temporal information such as videos (which are essentially a sequence of individual images) and blocks of text. The benefit of using CNNs is their ability to develop an internal representation of a two-dimensional image. This allows the model to learn position and scale in variant structures in the data, which is important when working with images.

To summarize, RNN is used for sequential data cases, and CNNs for interpreting visual data and data that does not come in a sequence.

#### 9) Explain what RNN problem is solved using LSTM and briefly describe how.


RNNs enable modelling time-dependent and sequential data tasks, such as stock market prediction, machine translation, text generation and many more.

RNNs suffer from the problem of vanishing gradients, which hampers learning of long data sequences. The gradients carry information used in the RNN parameter update and when the gradient becomes smaller and smaller, the parameter updates become insignificant which means no real learning is done.

LSTMs solve the problem using a unique additive gradient structure that includes direct access to the forget gate’s activations, enabling the network to encourage desired behaviour from the error gradient using frequent gates update on every time step of the learning process. 
LSTMs use a series of ‘gates’ which control how the information in a sequence of data comes into, is stored in and leaves the network. There are three gates in a typical LSTM; forget gate, input gate and output gate. These gates can be thought of as filters and are each their own neural network. We will explore them all in detail during the course of this article.
-  The forget gate controls what information in the cell state to forget, given new information than entered the network
- The input gate controls what new information will be encoded into the cell state, given the new input information
- The output gate controls what information encoded in the cell state is sent to the network as input in the following time step

    <img src = "Screen Shot 2020-12-03 at 7.36.20 PM.png" width="300">

In RNNs, the sum in the above equation is made from expressions with a similar behaviour that are likely to all be in [0,1] which causes vanishing gradients.
 
In LSTMs, however, the presence of the forget gate, along with the additive property of the cell state gradients, enables the network to update the parameter in such a way that the different sub gradients in the above equation do not necessarily agree and behave in a similar manner, making it less likely that all of the T gradients will vanish, or in other words, the series of functions does not converge to zero.
 
 
To summarize, RNNs suffer from vanishing gradients and caused by long series of multiplications of small values, diminishing the gradients and causing the learning process to become degenerate.RNNs suffer from exploding gradients affected from large gradient values and hampering the learning process.
LSTMs solve the problem using a unique additive gradient structure that includes direct access to the forget gate’s activations, enabling the network to encourage desired behaviour from the error gradient using frequent gates update on every time step of the learning process.

